<a href="https://colab.research.google.com/github/Kadabamon/traffic-sign-detection/blob/main/traffic_sign_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -------------------------------------------------------------
# INSTALL DEPENDENCIES & SETUP KAGGLE TOKEN
# -------------------------------------------------------------
!pip install -q kaggle tensorflow gradio scikit-learn matplotlib seaborn

import os
from google.colab import userdata

# Attempt to load kaggle.json from Colab Secrets
try:
    kaggle_json_content = userdata.get('KAGGLE_JSON')
    if kaggle_json_content:
        # Create .kaggle directory and write the file
        os.makedirs('/root/.kaggle', exist_ok=True)
        with open('/root/.kaggle/kaggle.json', 'w') as f:
            f.write(kaggle_json_content)
        os.chmod('/root/.kaggle/kaggle.json', 0o600)
        print("Kaggle API key configured from Colab Secrets.")
    else:
        # Fallback to manual upload if secret is not set
        from google.colab import files
        print("KAGGLE_JSON secret not found. Please add your kaggle.json content to Colab Secrets (name it 'KAGGLE_JSON').")
        print("⬆️ Upload your kaggle.json file now.")
        files.upload()  # Upload kaggle.json downloaded from Kaggle account
        print("Kaggle API key uploaded manually.")
except Exception as e:
    print(f"Error loading Kaggle API key from secrets: {e}")
    # Fallback to manual upload in case of any error with secrets
    from google.colab import files
    print("⬆️ Upload your kaggle.json file now.")
    files.upload()  # Upload kaggle.json downloaded from Kaggle account
    print("Kaggle API key uploaded manually.")

Error loading Kaggle API key from secrets: Secret KAGGLE_JSON does not exist.
⬆️ Upload your kaggle.json file now.


Saving kaggle.json to kaggle.json
Kaggle API key uploaded manually.


In [ ]:
# -------------------------------------------------------------
# SETUP KAGGLE API (CONTINUED)
# -------------------------------------------------------------
import os

kaggle_dir = os.path.expanduser("~/.kaggle")
kaggle_json_path_in_home = os.path.join(kaggle_dir, "kaggle.json")
kaggle_json_path_in_content = "/content/kaggle.json"

# Ensure ~/.kaggle directory exists
os.makedirs(kaggle_dir, exist_ok=True)

# Check if kaggle.json was manually uploaded and move it if necessary
if not os.path.exists(kaggle_json_path_in_home) and os.path.exists(kaggle_json_path_in_content):
    # If kaggle.json is in /content and not yet in ~/.kaggle, move it
    !mv "$kaggle_json_path_in_content" "$kaggle_json_path_in_home"
    print("Moved manually uploaded kaggle.json to ~/.kaggle/")

# Set permissions for kaggle.json if it exists in ~/.kaggle
if os.path.exists(kaggle_json_path_in_home):
    os.chmod(kaggle_json_path_in_home, 0o600)
    print("Kaggle API is set up successfully.")
else:
    print("Kaggle API key not found. Please ensure it's either in Colab Secrets (KAGGLE_JSON) or uploaded manually.")

Moved manually uploaded kaggle.json to ~/.kaggle/
Kaggle API is set up successfully.


In [ ]:
# -------------------------------------------------------------
# DOWNLOAD GTSRB DATASET
# -------------------------------------------------------------
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

# Unzip dataset
!unzip -q gtsrb-german-traffic-sign.zip -d gtsrb

print("Dataset downloaded and extracted.")

Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
 95% 582M/612M [00:04<00:00, 52.1MB/s]
100% 612M/612M [00:04<00:00, 152MB/s] 
Dataset downloaded and extracted.


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

IMG_SIZE = 224
BATCH_SIZE = 32

# Load training dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "gtsrb/Train",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

# Load test dataset manually as it often has a flat structure and .ppm files
# and image_dataset_from_directory might not handle it directly.

# Function to parse image and label
def parse_image(img_path, label):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    return img, label

# Read Test.csv to get file paths and labels
test_df = pd.read_csv("gtsrb/Test.csv")
test_image_paths = [os.path.join("gtsrb", test_df['Path'][i]) for i in range(len(test_df))]
test_labels = test_df['ClassId'].values

# Create a TensorFlow dataset from paths and labels
test_ds = tf.data.Dataset.from_tensor_slices((test_image_paths, test_labels))
test_ds = test_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)

print("Number of classes:", NUM_CLASSES)
print("Class names:", class_names[:10], "...")
print(f"Successfully loaded {tf.data.experimental.cardinality(train_ds).numpy() * BATCH_SIZE} training images.")
print(f"Successfully loaded {tf.data.experimental.cardinality(test_ds).numpy() * BATCH_SIZE} test images.")

Found 39209 files belonging to 43 classes.
Number of classes: 43
Class names: ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17'] ...
Successfully loaded 39232 training images.
Successfully loaded 12640 test images.


In [ ]:
# -------------------------------------------------------------
# BUILD TRANSFER LEARNING MODEL
# -------------------------------------------------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.4)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide (TrueDivide)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract (Subtract)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 43)             │        55,083 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,313,067 (8.82 MB)

 Trainable params: 55,083 (215.17 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# -------------------------------------------------------------
# CREATE VALIDATION SPLIT
# -------------------------------------------------------------
VAL_SPLIT = 0.2

train_size = int((1 - VAL_SPLIT) * len(train_ds))
val_size = len(train_ds) - train_size

train_data = train_ds.take(train_size)
val_data = train_ds.skip(train_size)

print("Train batches:", train_size)
print("Val batches:", val_size)


Train batches: 980
Val batches: 246


In [ ]:
# -------------------------------------------------------------
# DATA AUGMENTATION & PREFETCHING
# -------------------------------------------------------------
AUTOTUNE = tf.data.AUTOTUNE

data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)
])

train_data = train_data.map(lambda x, y: (data_aug(x, training=True), y))

train_data = train_data.prefetch(AUTOTUNE)
val_data = val_data.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


In [ ]:
# -------------------------------------------------------------
# INITIAL TRAINING
# -------------------------------------------------------------
EPOCHS = 10

history = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=val_data
)

Epoch 1/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 421s 429ms/step - accuracy: 0.9721 - loss: 0.0908 - val_accuracy: 0.9824 - val_loss: 0.0588
Epoch 2/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 420s 428ms/step - accuracy: 0.9759 - loss: 0.0744 - val_accuracy: 0.9847 - val_loss: 0.0497
Epoch 3/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 436s 422ms/step - accuracy: 0.9815 - loss: 0.0600 - val_accuracy: 0.9867 - val_loss: 0.0425
Epoch 4/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 413s 421ms/step - accuracy: 0.9846 - loss: 0.0493 - val_accuracy: 0.9902 - val_loss: 0.0311
Epoch 5/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 410s 419ms/step - accuracy: 0.9867 - loss: 0.0441 - val_accuracy: 0.9915 - val_loss: 0.0284
Epoch 6/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 419s 427ms/step - accuracy: 0.9880 - loss: 0.0374 - val_accuracy: 0.9930 - val_loss: 0.0247
Epoch 7/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 411s 420ms/step - accuracy: 0.9911 - loss: 0.0317 - val_accuracy: 0.9921 - val_loss: 0.0237
Epoch 8/10
980/980 ━━━━━━━━━━━━━━━━━━━━ 417s 425ms/step - accuracy: 0.9922 -

In [ ]:
# -------------------------------------------------------------
# FINE-TUNE MODEL
# -------------------------------------------------------------
base_model.trainable = True  # unfreeze full model

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-6),  # Reduced learning rate for fine-tuning
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

EPOCHS_FINE = 8

history_fine = model.fit(
    train_data,
    epochs=EPOCHS_FINE,
    validation_data=val_data
)

Epoch 1/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 479s 439ms/step - accuracy: 0.9940 - loss: 0.0205 - val_accuracy: 0.9971 - val_loss: 0.0126
Epoch 2/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 408s 416ms/step - accuracy: 0.9958 - loss: 0.0161 - val_accuracy: 0.9968 - val_loss: 0.0122
Epoch 3/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 415s 423ms/step - accuracy: 0.9963 - loss: 0.0156 - val_accuracy: 0.9966 - val_loss: 0.0125
Epoch 4/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 427s 436ms/step - accuracy: 0.9961 - loss: 0.0144 - val_accuracy: 0.9966 - val_loss: 0.0120
Epoch 5/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 412s 420ms/step - accuracy: 0.9966 - loss: 0.0142 - val_accuracy: 0.9966 - val_loss: 0.0117
Epoch 6/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 417s 426ms/step - accuracy: 0.9964 - loss: 0.0142 - val_accuracy: 0.9966 - val_loss: 0.0118
Epoch 7/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 405s 413ms/step - accuracy: 0.9963 - loss: 0.0143 - val_accuracy: 0.9967 - val_loss: 0.0116
Epoch 8/8
980/980 ━━━━━━━━━━━━━━━━━━━━ 402s 410ms/step - accuracy: 0.9957 - loss: 0

In [ ]:
# -------------------------------------------------------------
# EVALUATE MODEL
# -------------------------------------------------------------
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

print("Classification Report:")
print(classification_report(y_true, y_pred))


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model.save("gtsrb_mobilenetv2_model.keras")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
# -------------------------------------------------------------
# GRADIO DEMO
# -------------------------------------------------------------
import gradio as gr
import numpy as np
from PIL import Image

def predict(img):
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img)
    img = np.expand_dims(img, 0)
    preds = model.predict(img)
    idx = np.argmax(preds)
    return f"Predicted: {class_names[idx]} (Confidence: {np.max(preds):.3f})"

gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Road Sign Recognition"
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae4eaa83cc65e83953.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
